<a href="https://colab.research.google.com/github/Gowrish21S/DL_assignment/blob/main/TIC_tac_Toe_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pickle

BOARD_ROWS = 3
BOARD_COLS = 3


# --- State Class (No Change from previous Q-Learning version) ---
class State:
    def __init__(self, p1, p2):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.p1 = p1
        self.p2 = p2
        self.isEnd = False
        self.boardHash = None
        self.playerSymbol = 1
        self.current_player = p1

    def getHash(self):
        self.boardHash = str(self.board.reshape(BOARD_COLS * BOARD_ROWS))
        return self.boardHash

    def winner(self):
        # row
        for i in range(BOARD_ROWS):
            if sum(self.board[i, :]) == 3:
                self.isEnd = True
                return 1
            if sum(self.board[i, :]) == -3:
                self.isEnd = True
                return -1
        # col
        for i in range(BOARD_COLS):
            if sum(self.board[:, i]) == 3:
                self.isEnd = True
                return 1
            if sum(self.board[:, i]) == -3:
                self.isEnd = True
                return -1
        # diagonal
        diag_sum1 = sum([self.board[i, i] for i in range(BOARD_COLS)])
        diag_sum2 = sum([self.board[i, BOARD_COLS - i - 1] for i in range(BOARD_COLS)])

        if abs(diag_sum1) == 3 or abs(diag_sum2) == 3:
            self.isEnd = True
            if diag_sum1 == 3 or diag_sum2 == 3:
                return 1
            else:
                return -1

        # tie
        if len(self.availablePositions()) == 0:
            self.isEnd = True
            return 0

        # not end
        self.isEnd = False
        return None

    def availablePositions(self):
        positions = []
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if self.board[i, j] == 0:
                    positions.append((i, j))
        return positions

    def updateState(self, position):
        self.board[position] = self.playerSymbol
        self.playerSymbol = -1 if self.playerSymbol == 1 else 1
        self.current_player = self.p2 if self.playerSymbol == -1 else self.p1

    def giveReward(self, winner):
        if winner == 1:
            return 1
        elif winner == -1:
            return 1
        else:
            return 0.5

    def reset(self):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.boardHash = None
        self.isEnd = False
        self.playerSymbol = 1
        self.current_player = self.p1

    def play(self, rounds=100, exp_decay_rate=0.9999):
        for i in range(rounds):
            if i % 1000 == 0:
                print("Rounds {} | P1 Exp Rate: {:.4f}".format(i, self.p1.exp_rate))

            self.p1.update_exp_rate(exp_decay_rate)
            self.p2.update_exp_rate(exp_decay_rate)

            self.p1.reset_history()
            self.p2.reset_history()
            self.reset() # Reset the board for a new game

            while not self.isEnd:
                player_making_move = self.current_player # The player whose turn it is
                player_making_move_symbol = self.playerSymbol # Symbol of player making move

                positions_before_move = self.availablePositions()
                if not positions_before_move: # Should ideally be caught by winner() tie
                    self.isEnd = True
                    continue

                # Player chooses an action
                action = player_making_move.chooseAction(positions_before_move, self.board, player_making_move_symbol)

                # Store the state and action *before* the move is made, in the player's history
                player_making_move.addState(self.getHash(), action)

                # Make the move on the board
                self.updateState(action) # This internally flips self.playerSymbol and self.current_player

                # Check for game end after the move
                win = self.winner()

                # Get the hash and available positions for the *next* state (after the move)
                next_state_hash = self.getHash()
                next_state_positions = self.availablePositions()

                if win is not None:
                    # Game ended. The player who just moved (player_making_move) receives the reward.
                    reward_for_player = 0
                    if win == player_making_move_symbol: # The player who just moved won
                        reward_for_player = 1
                    elif win == 0: # It's a tie
                        reward_for_player = 0.5
                    # If player_making_move lost (win == -player_making_move_symbol), reward remains 0.

                    # Update the Q-table for the player who just moved (terminal state, so next_max_q_value is 0)
                    player_making_move.updateQTable(reward_for_player, next_max_q_value=0)

                    # Although not strictly part of this bug fix, in a more complete implementation,
                    # the opponent's last move (if any) might also need to be updated with terminal rewards.
                    # For simplicity and focusing on the IndexError, we primarily update the current mover.

                    self.reset() # Reset the board for the next round
                    break

                else:
                    # Game continues. Reward for the player who just moved is 0 for non-terminal states.
                    # Calculate the maximum expected Q-value for the *next* state (s') from the perspective
                    # of the player who just moved (player_making_move) using their own Q-table.
                    next_max_q_value_for_update = player_making_move.get_max_q_for_state(next_state_hash, next_state_positions)

                    # Update the Q-table for the player who just moved
                    player_making_move.updateQTable(0, next_max_q_value_for_update)

    def play2(self):
        while not self.isEnd:
            # Player 1 (AI)
            positions = self.availablePositions()
            p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol)

            self.updateState(p1_action)
            self.showBoard()

            win = self.winner()
            if win is not None:
                if win == 1:
                    print(self.p1.name, "wins!")
                elif win == 0:
                    print("tie!")
                else:
                    print(self.p2.name, "wins!")
                self.reset()
                break

            # Player 2 (Human)
            positions = self.availablePositions()
            p2_action = self.p2.chooseAction(positions)

            self.updateState(p2_action)
            self.showBoard()

            win = self.winner()
            if win is not None:
                if win == -1:
                    print(self.p2.name, "wins!")
                else:
                    print("tie!")
                self.reset()
                break

    def showBoard(self):
        for i in range(0, BOARD_ROWS):
            print('-------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = 'x'
                elif self.board[i, j] == -1:
                    token = 'o'
                else:
                    token = ' '
                out += token + ' | '
            print(out)
        print('-------------')


# --- QLearningPlayer Class (Modified) ---
class QLearningPlayer:
    def __init__(self, name, symbol, exp_rate=1.0, exp_min=0.01):
        self.name = name
        self.symbol = symbol
        self.history = []
        self.lr = 0.2
        self.exp_rate = exp_rate
        self.exp_min = exp_min
        self.decay_gamma = 0.9
        self.q_table = {}

    def getHash(self, board):
        return str(board.reshape(BOARD_COLS * BOARD_ROWS))

    def chooseAction(self, positions, current_board, symbol):
        current_hash = self.getHash(current_board)
        if np.random.uniform(0, 1) <= self.exp_rate:
            # Exploration: choose a random action
            idx = np.random.choice(len(positions))
            action = positions[idx]
        else:
            # Exploitation: choose action with max Q-value
            value_max = -np.inf # Initialize with negative infinity
            action = None

            for p in positions:
                q_value = self.q_table.get((current_hash, p), 0)
                if q_value > value_max:
                    value_max = q_value
                    action = p

            # Fallback for when all Q-values are 0 or less (e.g., initial state) or no positions
            if action is None and positions:
                idx = np.random.choice(len(positions))
                action = positions[idx]
            elif action is None and not positions: # If no positions are available, this case shouldn't be reached ideally
                # This indicates an issue upstream or a game end condition not fully handled
                raise ValueError("No available positions for action selection.")

        return action

    def addState(self, state_hash, action): # Removed next_q_value parameter
        # Stores the state and action taken by this player
        self.history.append({'state_hash': state_hash, 'action': action})

    def updateQTable(self, reward, next_max_q_value): # Renamed for clarity: next_q_value is max(Q(s',a'))
        st_act_info = self.history.pop() # Retrieve the last state-action pair from history
        state_action = (st_act_info['state_hash'], st_act_info['action'])
        old_q = self.q_table.get(state_action, 0)

        # Q-Learning Update Rule: Q(s,a) = Q(s,a) + lr * [R + decay_gamma * max(Q(s',a')) - Q(s,a)]
        new_q = old_q + self.lr * (reward + self.decay_gamma * next_max_q_value - old_q)
        self.q_table[state_action] = new_q

    # --- NEW METHOD: Get max Q-value for a given state ---
    def get_max_q_for_state(self, state_hash, available_positions):
        if not available_positions:
            return 0 # If no moves are possible from this state, its value is 0 (terminal or no moves left)
        max_q = -np.inf
        for p in available_positions:
            q_val = self.q_table.get((state_hash, p), 0)
            if q_val > max_q:
                max_q = q_val
        return max_q if max_q != -np.inf else 0 # Return 0 if all Q-values are -inf (e.g., brand new state)

    def update_exp_rate(self, decay_rate):
        self.exp_rate = max(self.exp_min, self.exp_rate * decay_rate)

    def reset_history(self):
        self.history = []

    def savePolicy(self):
        fw = open('q_policy_' + str(self.name), 'wb')
        pickle.dump(self.q_table, fw)
        fw.close()

    def loadPolicy(self, file):
        fr = open(file, 'rb')
        self.q_table = pickle.load(fr)
        fr.close()


class HumanPlayer:
    def __init__(self, name):
        self.name = name

    def chooseAction(self, positions):
        while True:
            try:
                row = int(input("Input your action row (0-2):"))
                col = int(input("Input your action col (0-2):"))
                action = (row, col)
                if action in positions:
                    return action
                else:
                    print("Invalid position. Try again.")
            except ValueError:
                print("Invalid input. Please enter a number.")
            except Exception:
                print("An error occurred.")

    def reset_history(self):
        pass


if __name__ == "__main__":
    # training
    # Start with exp_rate = 1.0 (100% exploration)
    p1 = QLearningPlayer("p1", symbol=1, exp_rate=1.0)
    p2 = QLearningPlayer("p2", symbol=-1, exp_rate=1.0)

    st = State(p1, p2)
    print("training...")
    # Use a small decay rate over many rounds to slowly reduce exploration
    # Example: 1.0 * (0.9999)^50000 approx 0.0067 (just above the min of 0.01)
    st.play(50000, exp_decay_rate=0.9999)

    p1.savePolicy()
    p2.savePolicy()

    # play with human
    # Set exp_rate=0 for deterministic, optimal play against the human
    p1 = QLearningPlayer("computer", symbol=1, exp_rate=0)
    try:
        p1.loadPolicy("q_policy_p1")
    except FileNotFoundError:
        print("Policy file not found. Run training first.")

    p2 = HumanPlayer("human")

    st = State(p1, p2)

    c ='y'
    while (c == 'y'):
        st.play2()
        print("---")
        print("If you wish to continue press 'y', else press 'n'")
        c = (input("Input your choice to continue:"))


training...
Rounds 0 | P1 Exp Rate: 1.0000
Rounds 1000 | P1 Exp Rate: 0.9048
Rounds 2000 | P1 Exp Rate: 0.8187
Rounds 3000 | P1 Exp Rate: 0.7408
Rounds 4000 | P1 Exp Rate: 0.6703
Rounds 5000 | P1 Exp Rate: 0.6065
Rounds 6000 | P1 Exp Rate: 0.5488
Rounds 7000 | P1 Exp Rate: 0.4966
Rounds 8000 | P1 Exp Rate: 0.4493
Rounds 9000 | P1 Exp Rate: 0.4066
Rounds 10000 | P1 Exp Rate: 0.3679
Rounds 11000 | P1 Exp Rate: 0.3329
Rounds 12000 | P1 Exp Rate: 0.3012
Rounds 13000 | P1 Exp Rate: 0.2725
Rounds 14000 | P1 Exp Rate: 0.2466
Rounds 15000 | P1 Exp Rate: 0.2231
Rounds 16000 | P1 Exp Rate: 0.2019
Rounds 17000 | P1 Exp Rate: 0.1827
Rounds 18000 | P1 Exp Rate: 0.1653
Rounds 19000 | P1 Exp Rate: 0.1496
Rounds 20000 | P1 Exp Rate: 0.1353
Rounds 21000 | P1 Exp Rate: 0.1224
Rounds 22000 | P1 Exp Rate: 0.1108
Rounds 23000 | P1 Exp Rate: 0.1002
Rounds 24000 | P1 Exp Rate: 0.0907
Rounds 25000 | P1 Exp Rate: 0.0821
Rounds 26000 | P1 Exp Rate: 0.0743
Rounds 27000 | P1 Exp Rate: 0.0672
Rounds 28000 | P1 Exp